# Police Tweets collection


## Imports

In [4]:
from __future__ import unicode_literals

import tweepy
import configparser
import pandas as pd
from datetime import datetime
import json
from datetime import datetime, timedelta, timezone
import time
import os
import pandas as pd

## Variables

In [2]:
data_dir = "./../../../data/"
twitter_data_dir = data_dir + "twitter/"

## Keys

In [3]:
config = configparser.ConfigParser(interpolation=None)
config.read(twitter_data_dir + 'keys/config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

bearer_token = config['twitter']['bearer_token']

## Authentication

In [4]:
# Access to API

def authenticate_twitter(api_key, api_key_secret, access_token, access_token_secret):
    auth = tweepy.OAuthHandler(api_key, api_key_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

In [5]:
api = authenticate_twitter(api_key, api_key_secret, access_token, access_token_secret)

In [ ]:
class MyStream(tweepy.StreamingClient):
    
    tweets = []
    limit = 1000

    def on_connect(self):
        print("Connected")

    def on_tweet(self, tweet):
        self.tweets.append(tweet)
        if len(self.tweets) == self.limit:
            self.disconnect()
            return

In [ ]:
stream = MyStream(bearer_token)

In [ ]:
stream.add_rules(tweepy.StreamRule("RPP"))

In [ ]:
stream.filter()

In [ ]:
stream.tweets

In [ ]:
list_tweets = [tweet for tweet in stream_tweet.tweets]

## Account tweets and search tweets 

In [3]:
# initialize min_id = -1 and update it for continuse collecting 
df_min_id = pd.read_csv("name_min_id.csv")
df_min_id

,Name,Handler,Min_id_search,Min_id_account
0,Rīgas pašvaldības policija,@RigasPP,1661733283212988427,1661708433609965568
1,Préfecture de Police,@prefpolice,1661736653805981698,1661717604870897665
2,Polizei Berlin Einsatz,@PolizeiBerlin_E,1661735417706848257,1661390661407391744
3,Polizei Berlin,@polizeiberlin,1661736926964236292,1661678250903347200
4,City of London Police,@CityPolice,1661739025554874369,1661728267253211139
5,Metropolitan Police,@metpoliceuk,1661740733043441671,1661733305535057925
6,Met Contact Centre,@MetCC,1661750859200622592,1661733890611085318
7,Police Municipale de Paris,@PMdeParis,1661746030310985728,1661031739181989889
8,Polizia Roma Capitale,@PLRomaCapitale,1661747097820090373,1432357667528392704
9,Questura Di Roma,@QuesturaDiRoma,1661623067905253380,1611247422755192835


In [7]:
#3200 tweets
for i in range(len(df_min_id)):
    user = df_min_id.Handler[i][1:]
    since_id = df_min_id.Min_id_account[i]
    print(user)
    #maximum retrive 3200 from one user
    tweets = tweepy.Cursor(api.user_timeline,
                               screen_name=user,
                               since_id = since_id,
                               count=200,
                               tweet_mode='extended').items(13200)
    list_tweets = [tweet._json for tweet in tweets]
    
    if list_tweets:
        most_recent_id = list_tweets[0]['id']
        df_min_id.loc[i,'Min_id_account'] = most_recent_id
        df_min_id.to_csv("name_min_id.csv",index=False)


        print("Retrieved {} tweets".format(len(list_tweets)))
        date = datetime.today().strftime("%d-%m-%Y")
        collected_tweets = {"tweets":list_tweets}
        tweets_storage_file = twitter_data_dir + "added_account_tweets2/collected_"+ str(len(list_tweets)) + "_tweets_" + user + date +".json"
        with open(tweets_storage_file, "w", encoding='utf-8') as f:
            json.dump(collected_tweets, f, ensure_ascii=False, indent=4)

RigasPP
Retrieved 9 tweets
prefpolice
Retrieved 8 tweets
PolizeiBerlin_E
Retrieved 18 tweets
polizeiberlin
Retrieved 17 tweets
CityPolice
Retrieved 22 tweets
metpoliceuk
Retrieved 43 tweets
MetCC
Retrieved 127 tweets
PMdeParis
Retrieved 3 tweets
PLRomaCapitale
QuesturaDiRoma
policiademadrid
Retrieved 24 tweets
Policja_KSP
Retrieved 14 tweets
POL_Amsterdam
Retrieved 8 tweets
MP_Praha
psplisboa
Retrieved 3 tweets
polisen_sthlm
Retrieved 54 tweets
LPDWien
Retrieved 62 tweets
PoliceBern
Retrieved 7 tweets
KobenhavnPoliti
Retrieved 16 tweets
HelsinkiPoliisi
Retrieved 13 tweets
oslopolitiops
Retrieved 61 tweets
PUzgFeed


In [8]:
# last 10 days search with hander and name
# query = ["Préfecture de Police OR @prefpolice", "Rīgas pašvaldības policija OR @RigasPP"]

for i in range(len(df_min_id)):
        query = df_min_id.Handler[i]  + " OR ("  + df_search.Name[i] + ")"
        # query = df_search.Name[i]
        min_id = df_min_id.Min_id_search[i]
        print(query)
        
        tweets = tweepy.Cursor(api.search_tweets,
                               q=query,
                               count=100,
                               since_id = min_id,
                               tweet_mode='extended').items(50000)
        list_tweets = [tweet._json for tweet in tweets]
        
        if list_tweets:
            most_recent_id = list_tweets[0]['id']
            laste_retrived_date = datetime.strptime(list_tweets[-1]['created_at'][4:],'%b %d %H:%M:%S %z %Y')  
            
            df_min_id.loc[i,'Min_id_search'] = most_recent_id
            df_min_id.to_csv("name_min_id.csv",index=False)
            
            collected_tweets = {"tweets":list_tweets}
            tweets_storage_file = twitter_data_dir + "added_search_tweets2/"+ str(len(list_tweets)) +"items_"+ query + "_" + str(most_recent_id) + "_" + str(laste_retrived_date) + ".json"
            with open(tweets_storage_file, "w", encoding='utf-8') as f:
                    json.dump(collected_tweets, f, ensure_ascii=False, indent=4)
                    print("=== saved {} search on {}, with last date {} and max_id {} , ".format(len(list_tweets),query,laste_retrived_date,most_recent_id))
        

@RigasPP OR (Rīgas pašvaldības policija)
=== saved 220 search on @RigasPP OR (Rīgas pašvaldības policija), with last date 2023-05-20 10:25:29+00:00 and max_id 1661733283212988427 , 
@prefpolice OR (Préfecture de Police)
=== saved 4742 search on @prefpolice OR (Préfecture de Police), with last date 2023-05-20 10:01:38+00:00 and max_id 1661736653805981698 , 
@PolizeiBerlin_E OR (Polizei Berlin Einsatz)
=== saved 4213 search on @PolizeiBerlin_E OR (Polizei Berlin Einsatz), with last date 2023-05-20 10:02:46+00:00 and max_id 1661735417706848257 , 
@polizeiberlin OR (Polizei Berlin)


Rate limit reached. Sleeping for: 748


=== saved 12670 search on @polizeiberlin OR (Polizei Berlin), with last date 2023-05-20 09:57:44+00:00 and max_id 1661736926964236292 , 
@CityPolice OR (City of London Police)
=== saved 1169 search on @CityPolice OR (City of London Police), with last date 2023-05-20 10:17:06+00:00 and max_id 1661739025554874369 , 
@metpoliceuk OR (Metropolitan Police)


Rate limit reached. Sleeping for: 727
Rate limit reached. Sleeping for: 721
Rate limit reached. Sleeping for: 732


=== saved 50000 search on @metpoliceuk OR (Metropolitan Police), with last date 2023-05-22 09:15:52+00:00 and max_id 1661740733043441671 , 
@MetCC OR (Met Contact Centre)
=== saved 1622 search on @MetCC OR (Met Contact Centre), with last date 2023-05-21 15:17:02+00:00 and max_id 1661750859200622592 , 
@PMdeParis OR (Police Municipale de Paris)
=== saved 305 search on @PMdeParis OR (Police Municipale de Paris), with last date 2023-05-21 15:55:57+00:00 and max_id 1661746030310985728 , 
@PLRomaCapitale OR (Polizia Roma Capitale)
=== saved 175 search on @PLRomaCapitale OR (Polizia Roma Capitale), with last date 2023-05-21 15:13:11+00:00 and max_id 1661747097820090373 , 
@QuesturaDiRoma OR (Questura Di Roma)
=== saved 40 search on @QuesturaDiRoma OR (Questura Di Roma), with last date 2023-05-21 18:04:09+00:00 and max_id 1661623067905253380 , 
@policiademadrid OR (Policía Municipal de Madrid)
=== saved 1878 search on @policiademadrid OR (Policía Municipal de Madrid), with last date 2023-05-2

## profile of accounts

In [97]:
#profile info
from pprint import pprint

from nitter_scraper import NitterScraper
list_tweets = []
counter = 0

with NitterScraper(host="0.0.0.0", port=8008) as nitter:
    for username in df.Handler:
        profile = nitter.get_profile(username)
        # print("serialize to json\n")
        # print(profile.json(indent=4))
        # print("serialize to a dictionary\n")
        # pprint(profile.dict())
        list_tweets += [json.loads(profile.json())]
        counter += 1

    # collected_tweets = {"profiles":list_tweets}
    tweets_storage_file = twitter_data_dir + "tweets/profile_final/profile_" + str(counter)+"accounts" +".json"
    with open(tweets_storage_file, "w", encoding='utf-8') as f:
        json.dump(list_tweets, f, ensure_ascii=False, indent=4)

2023-04-26 16:12:22.604 | INFO     | nitter_scraper.nitter:_get_client:31 - Docker connection successful.
2023-04-26 16:12:23.926 | INFO     | nitter_scraper.nitter:start:155 - Running container recursing_nobel 597950758c.
2023-04-26 16:12:48.385 | INFO     | nitter_scraper.nitter:stop:159 - Stopping container recursing_nobel 597950758c.
2023-04-26 16:12:53.566 | INFO     | nitter_scraper.nitter:stop:162 - Container recursing_nobel 597950758c Destroyed.


## Clean search tweets


In [7]:
search_data_dir = twitter_data_dir + "added_search_tweets2/"
all_search_files = [f for f in os.listdir(search_data_dir) if f.endswith('.json')]

In [6]:
def check_search(key_word,file_name,df,checkRT):
    try:
        old_df = pd.read_csv(file_name, low_memory=False)
    except pd.errors.EmptyDataError:
        old_df = pd.DataFrame()
        
    if checkRT:
        df = df[df.retweeted_status.map(lambda x: all([i in x['full_text'] for i in key_word]))]
        
    else:
        df = df[df.full_text.map(lambda x: all([i in x for i in key_word]))]
        
    new_df = pd.concat([old_df,df], ignore_index=True)
    new_df.drop_duplicates("id").reset_index(drop=True).to_csv(file_name,index=False)

In [ ]:
# go through each search file to check matched @user or account_name string

for i in range(len(all_search_files)):
    f = all_search_files[i]
    
    with open(search_data_dir+f) as fp:
        data = json.load(fp)['tweets']
        df_tweets = pd.DataFrame(data=data)
        if len(df_tweets) == 0: continue 
        
        # contian full text
        df_tweets = df_tweets[df_tweets.full_text.notna()]
        
        #  normalise text(accent), Lower case the tweet text
        df_tweets.full_text = df_tweets.full_text.map(lambda x: unidecode(x.lower()))
        
        # remove url
        df_tweets.full_text = df_tweets.full_text.map(lambda x: re.sub('http[s]?://\S+', '', x))
        
    for i in range(len(df)):
        name = df.loc[i,'Name'].lower()
        name_en = GoogleTranslator(source='auto', target='en').translate(name)
        handler = df.loc[i,'Handler'].lower()
        if len(df_tweets) == 0: continue 
        # check if user is not account itself
        df_tweets = df_tweets[df_tweets.user.map(lambda x: x['screen_name'] != handler[1:])]
        
        
        #1.orginial tweets contian: @user | name
        if 'retweeted_status' in df_tweets.keys():
            df_orig = df_tweets[df_tweets['retweeted_status'].isna() & ~(df_tweets.is_quote_status)]
        else: 
            df_orig = df_tweets[~(df_tweets.is_quote_status)]
        if len(df_orig) >0:
            checkRT = False
            check_search(name.split(),"tweets/merged_search/orginal_"+name+".csv",df_orig,checkRT)
            check_search(name_en.split(),"tweets/merged_search/orginal_"+name+".csv",df_orig,checkRT)
            check_search([handler],"tweets/merged_search/orginal_"+handler+".csv",df_orig,checkRT)

        # 2. quote(reply) contian: @user | name
        df_quote = df_tweets[df_tweets.is_quote_status]
        if len(df_quote)>0:
            checkRT = False
            check_search(name.split(),"tweets/merged_search/quote_"+name+".csv",df_quote,checkRT)
            check_search(name_en.split(),"tweets/merged_search/quote_"+name+".csv",df_quote,checkRT)
            check_search([handler],"tweets/merged_search/quote_"+handler+".csv",df_quote,checkRT)


        
        #3. retweets from others contian: @user | contain string
        if 'retweeted_status' in df_tweets.keys():
            df_rt = df_tweets[df_tweets['retweeted_status'].notna()]

            df_rt_non_acc = df_rt[df_rt['retweeted_status'].map(lambda x: x['user']['screen_name']!= handler[1:])]
            
            if len(df_rt_non_acc) > 0:
                checkRT = True
                check_search(name.split(),"tweets/merged_search/rt_other_rt_"+name+".csv", df_rt_non_acc, checkRT)
                check_search(name_en.split(),"tweets/merged_search/rt_other_rt_"+name+".csv", df_rt_non_acc, checkRT)
                check_search([handler],"tweets/merged_search/rt_other_rt_"+handler+".csv", df_rt_non_acc, checkRT)


        

## Merge search  tweets

In [ ]:
merged_search_dir = "tweets/merged_search/"
merged_search_files = [f for f in os.listdir(merged_search_dir) if f.endswith('.csv')]

account_data_dir = "tweets/collected_account/" 
all_account_files = [f for f in os.listdir(account_data_dir) if f.endswith('.json')]

df = pd.read_csv('city-label.csv')

In [ ]:
# assign files to each account
def assign_files(df,columns_name,all_files):
    df[columns_name] = [[] for _ in range(len(df))]
    
    for i in range(len(df)):
        handler = df.loc[i,"Handler"][1:].lower()
        name = df.loc[i,"Name"][1:].lower()
        files = []
        for f in all_files:
            if (name in f.lower()) | (handler in f.lower()):
                files += [f] 
        df.at[i,columns_name] = files
        
    df_city = df.groupby("City")[columns_name].sum().reset_index()
    return df_city
        
df_city_search = assign_files(df,'search_files',merged_search_files)
df_city_account = assign_files(df,'account_files',all_account_files)

In [ ]:
def merge_to_city(df_city,tweet_type,to_save_data_dir,open_date_dir,file_column):
    for i in range(len(df_city)):
        city = df_city.loc[i,'City']
        df_collected = pd.DataFrame()

        for f in df_city.loc[i,file_column]:
            file_path = open_date_dir+f

            if f.split('.')[-1] == "json":
                with open(file_path) as fp:
                    data = json.load(fp)["tweets"]
                    df_tweets = pd.DataFrame(data=data)
                    
            elif f.split('.')[-1] == "csv":
                try:
                    df_tweets = pd.read_csv(file_path, low_memory=False)
                except pd.errors.EmptyDataError:
                    df_tweets = pd.DataFrame()
                

            df_collected = pd.concat([df_collected,df_tweets], ignore_index=True)

        df_collected = df_collected.drop_duplicates("id").reset_index(drop=True)
        df_collected.to_csv(to_save_data_dir+city+"_"+tweet_type+"_"+str(len(df_collected))+".csv", index=False)

merge_to_city(df_city_search,'search',"tweets/merged_search_city21/",merged_search_dir,'search_files')   
merge_to_city(df_city_account,'account',"tweets/merged_account_city2/",account_data_dir,'account_files')     


In [ ]:
m1 = "tweets/merged_account_city/"
m2 = "tweets/merged_account_city2/" 
all_account = [m1+f for f in os.listdir(m1) if f.endswith('.csv')] + [m2+f for f in os.listdir(m2) if f.endswith('.csv')]

In [10]:
df_basic_info = pd.read_json('profile_23accounts.json')

In [12]:
create_date = []
for i in range(len(df_basic_info)):
    account_start_date = api.get_user(screen_name=df_basic_info.username[i])
    create_date +=  [account_start_date.created_at]
    
df_basic_info["Created_date"] = create_date

In [15]:
df_basic_info.to_csv('basic_info.csv',index=False) 

1. all the post tweets from the user
2. tweets search 

In [56]:
#long string

# queries = [["Polizei Berlin Einsatz","@PolizeiBerlin_E"], "Polizei Berlin Prävention OR @PolizeiBerlin_P", "Polizei Berlin OR @polizeiberlin", "Metropolitan Police Events OR @MetPoliceEvents", "City of London Police OR @CityPolice", "Metropolitan Police OR @metpoliceuk", "Police Municipale de Paris OR @PMdeParis", "PoliziaRomaCapitale OR @PLRomaCapitale", "QuesturaDiRoma OR @QuesturaDiRoma", "Policía Municipal de Madrid OR @policiademadrid", "Policja Warszawa OR @Policja_KSP", "Poliția Română OR @_PolitiaRomana_", "Politie Eenheid Amsterdam OR @POL_Amsterdam"]
queries = [["Polizei Berlin", "@polizeiberlin"]]



for q in queries:

    # query = q[0]+ " OR " + q[1] 
    query = q[1]
    print(query)
    
    # set date 
    account_start_date = api.get_user(screen_name=q[1][1:])
    print(q[1][1:])
    date_create =  account_start_date.created_at

    date_now = datetime.now(timezone.utc)
    
    weeks_number = (date_now - date_create).days//7
    max_id = date_now
    list_tweets = []

    for i in range(weeks_number+1):
        # each time back to one week
        since_id = max_id - timedelta(days=14)
        print(since_id)


        tweets = tweepy.Cursor(api.search_tweets,
                           q=query,
                           count=200,
                           tweet_mode='extended').items(3200)
        list_tweets = list_tweets + [tweet._json for tweet in tweets]
        print("Retrieved {} tweets".format(len(list_tweets)))
        max_id = since_id


    collected_tweets = {"tweets":list_tweets}
    tweets_storage_file = twitter_data_dir + "tweets/collected_tweets_" + since_id + query + ".json"
    with open(tweets_storage_file, "w", encoding='utf-8') as f:
            json.dump(collected_tweets, f, ensure_ascii=False, indent=4)

    
    
    
    


@polizeiberlin
polizeiberlin
2023-03-30 05:02:40.023856+00:00
Retrieved 0 tweets
2023-03-23 05:02:40.023856+00:00
Retrieved 0 tweets
2023-03-16 05:02:40.023856+00:00
Retrieved 0 tweets
2023-03-09 05:02:40.023856+00:00
Retrieved 0 tweets
2023-03-02 05:02:40.023856+00:00
Retrieved 0 tweets
2023-02-23 05:02:40.023856+00:00
Retrieved 0 tweets
2023-02-16 05:02:40.023856+00:00


KeyboardInterrupt: 

In [70]:
for query in queries:
    query = 'Polizei Berlin since:2021-11-03 until:2021-12-03 '
    tweets = tweepy.Cursor(api.search_tweets,
                           q=query,
                           count = 200,
                           tweet_mode='extended').items(3200)
    list_tweets = [tweet._json for tweet in tweets]
    collected_tweets = {"tweets":list_tweets}
    print("Retrieved {} tweets".format(len(list_tweets)))

Retrieved 0 tweets


In [6]:
# query = 'RigasPP OR (Rīgas pašvaldības policija)'

In [7]:
# tweets = tweepy.Cursor(api.search_tweets,
#                        q=query,
#                        since_id = date_since,
#                        tweet_mode='extended').items(numtweets)

In [63]:
# list_tweets = [tweet._json for tweet in tweets]

In [67]:
list_tweets

[{'created_at': 'Thu Apr 06 05:29:17 +0000 2023',
  'id': 1643848328906436608,
  'id_str': '1643848328906436608',
  'full_text': 'RT @FrKretschBuch: Mir erzählt Keiner mehr, dass das alles tagtäglich überall in #Deutschland zufällig passiert, weil "geflüchtete" Mensche…',
  'truncated': False,
  'display_text_range': [0, 140],
  'entities': {'hashtags': [{'text': 'Deutschland', 'indices': [81, 93]}],
   'symbols': [],
   'user_mentions': [{'screen_name': 'FrKretschBuch',
     'name': 'Die Kretsche',
     'id': 1160818849987645441,
     'id_str': '1160818849987645441',
     'indices': [3, 17]}],
   'urls': []},
  'metadata': {'iso_language_code': 'de', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1683402367,
   'id_str': '

In [10]:
collected_tweets = {"tweets":list_tweets}
print("Retrieved {} tweets".format(len(list_tweets)))

Retrieved 460 tweets


In [68]:
time_now = datetime.now()
time_now = time_now.strftime("%d-%m-%Y_%H-%M-%S")

tweets_storage_file = twitter_data_dir + "tweets/collected_tweets_" +time_now + ".json"

In [69]:
with open(tweets_storage_file, "w", encoding='utf-8') as f:
    json.dump(collected_tweets, f, ensure_ascii=False, indent=4)